In [ ]:
%load_ext autoreload
%autoreload 2

import gin
gin.enter_interactive_mode()

import ddsp.training

In [ ]:
data_provider = ddsp.training.data.TFRecordProvider(
    "/Volumes/euler/datasets/transfer4/transfer4.tfrecord*",
    frame_rate=50,
    centered=True,
    with_jukebox=True,
)
dataset = data_provider.get_batch(batch_size=1, shuffle=True, repeats=1)
batch = next(iter(dataset))

In [ ]:
data_provider = ddsp.training.data.TFRecordProvider(
    "/Users/vaclav/prog/thesis/data/datasets/violin3/violin3.tfrecord*",
    frame_rate=50,
    centered=True,
    with_jukebox=True,
)
dataset = data_provider.get_batch(batch_size=1, shuffle=True, repeats=1)
batch = next(iter(dataset))

In [ ]:
batch["loudness_db"]

In [ ]:
batch.keys()

In [ ]:
import thesis
from thesis.notebook_util import play_audio

In [ ]:
play_audio(batch["audio_16k"])

In [ ]:
import ddsp.training.jukebox
import torch
from ddsp.training.jukebox import vqvae

In [ ]:
decoded = ddsp.training.jukebox.vqvae.decode(
    [
        torch.as_tensor(batch["jukebox_indices_0"][:,:-1].numpy()).type(torch.IntTensor),
        torch.as_tensor(batch["jukebox_indices_1"][:,:-1].numpy()).type(torch.IntTensor),
        torch.as_tensor(batch["jukebox_indices_2"][:,:-1].numpy()).type(torch.IntTensor),
    ],
    start_level=0,
)

In [ ]:
play_audio(decoded)

In [ ]:
for i in range(10):
    indices = torch.as_tensor(batch["jukebox_indices_2"][:, :-1].numpy()).type(torch.IntTensor)
    rot = i
    indices = (indices + rot) % 2048

    decoded = ddsp.training.jukebox.vqvae.decode(
        [
            indices,
            # torch.as_tensor(batch["jukebox_indices_1"][:, :-1].numpy()).type(torch.IntTensor),
            # torch.as_tensor(batch["jukebox_indices_2"][:, :-1].numpy()).type(torch.IntTensor),
        ],
        start_level=2,
    )

    play_audio(decoded)

In [ ]:
play_audio(batch["audio_16k"])

In [ ]:
encodings_gt = vqvae.encode(torch.as_tensor(batch["audio_16k"].numpy()).reshape(1, -1, 1))

In [ ]:
encodings_gt[2][0].numpy()[300]

In [ ]:
batch["jukebox_indices_2"][0][300]

In [ ]:
decoded_gt = vqvae.decode(encodings_gt)
play_audio(decoded_gt)

In [ ]:
decoded = torch.nn.functional.embedding(
    encodings_gt[0],
    vqvae.bottleneck.level_blocks[0].k,
)[0].t()

In [ ]:
decoded.shape

In [ ]:
x = vqvae.bottleneck.decode(encodings_gt)

In [ ]:
torch.all(x[0] == decoded)

In [ ]:
ddsp.training.jukebox.embedding_lookup(
    [batch["jukebox_embeddings_0"], batch["jukebox_embeddings_1"], batch["jukebox_embeddings_2"]]
)[2].shape

In [ ]:
import tensorflow as tf

params = tf.convert_to_tensor(vqvae.bottleneck.level_blocks[0].k)
ids = batch["jukebox_indices_0"].astype(tf.int32)

x_tf = tf.nn.embedding_lookup(
    params, ids,
)

In [ ]:
x_tf.shape

In [ ]:
x_tf[0,:5,:5]

In [ ]:
x[0][0,:5,:5].t()